# 2. Сохранение и загрузка моделей: pickle и joblib

### ИНСТРУМЕНТЫ СЕРИАЛИЗАЦИИ: PICKLE

In [1]:
# В качестве модели, прогнозирующей целевую переменную, возьмём простейшую линейную регрессию, и обучим её на исходных данных:

from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

In [2]:
# Далее, когда мы получили обученную модель, нам необходимо сериализовать её, превратив объект Python в поток байтов. 
# Для этого импортируем модуль pickle и воспользуемся функцией dumps(), в которую нужно передать объект Python.

import pickle

# Производим сериализацию обученной модели
model = pickle.dumps(regressor)

print(type(model))
print(type(regressor))
## bytes
## sklearn.linear_model._base.LinearRegression

<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


In [3]:
# Давайте попробуем восстановить (десериализовать) объект Python. 
# Для этого в модуле pickle есть функция loads(), в которую нужно передать сериализованный объект (поток байтов).

# Производим десериализацию
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

In [4]:
# Сохраним сериализованный объект прямо в файл. Для этого в pickle есть функция dump() (без s на конце). 
# В неё необходимо передать имя файла или ссылку на открытый файл. Файл назовём myfile, его расширение — .pkl (формат данных pickle):

# Производим сериализацию и записываем результат в файл формата pkl
with open('myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

In [5]:
# Посмотрим на код, который восстанавливает (десериализует) обученную модель из файла myfile.pkl. 
# Для этого в pickle есть функция load() (без s на конце). В неё необходимо передать имя файла или ссылку на открытый файл.

# Производим десериализацию и извлекаем модель из файла формата pkl
with open('myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

In [6]:
# Убедимся, что методы и результаты предсказаний обученной модели и модели, загруженной из файла, совпадают:

# Проверяем, что все элементы массивов предсказаний совпадают между собой
all(regressor.predict(X) == regressor_from_bytes.predict(X))
## True
all(regressor.predict(X) == regressor_from_file.predict(X))
## True

True

In [7]:
# Как мы упоминали, у pickle есть ограничения. Например, мы не можем сериализовать лямбда-функции. Давайте посмотрим, что нам вернёт следующий код:

my_lambda = lambda x: x*2
with open('my_lambda.pkl', 'wb') as output:
    pickle.dump(my_lambda, output)
 
##"PicklingError: Can't pickle <function <lambda>"

PicklingError: Can't pickle <function <lambda> at 0x134d157e0>: attribute lookup <lambda> on __main__ failed

### СОХРАНЕНИЕ ПАЙПЛАЙНА

In [8]:
# Например, мы хотим сериализовать пайплайн, который включает в себя min-max-нормализацию 
# отбор пяти наиболее важных факторов на основе корреляции Пирсона. 
# Полученные в результате данные отправляются на вход модели линейной регрессии.

import pickle

from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x135a331c0>)),
                ('Linear', LinearRegression())])

In [9]:
# Сериализуем pipeline и записываем результат в файл
with open('my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [10]:
# Если сериализация завершилась успешно, то при инференсе модели мы сможем восстановить её из файла:

# Десериализуем pipeline из файла
with open('my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

In [11]:
# Проверим, что результаты исходного и десериализованного пайплайнов и идентичны:

# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

## True

True


In [13]:
# Предположим, мы хотим генерировать в данных новый признак, который является простым произведением первых трёх столбцов таблицы. 
# Давайте пропишем в методе transform() эти действия.

# Для работы такого трансформера нужны только исходные данные без дополнительных параметров, 
# поэтому методы __init__() и fit() остаются без изменений.
import numpy as np
from sklearn.base import TransformerMixin, BaseEstimator

class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

In [14]:
# Посмотрим, как работает наш кастомный трансформер. 
# Создадим объект трансформера, вызовем метод transform и посмотрим на результирующий размер таблицы.

# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

## Shape before transform: (442, 10)
## Shape after transform: (442, 11)

Shape before transform: (442, 10)
Shape after transform: (442, 11)


In [15]:
# Теперь давайте встроим этот трансформер в сам пайплайн — для этого достаточно добавить новый шаг в пайплайн.

# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x135a331c0>)),
                ('Linear', LinearRegression())])

In [16]:
# Наконец можно сериализовать полученный pipeline:

# Сериализуем pipeline и записываем результат в файл
with open('my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [17]:
# 2.5
# Десериализуйте полученный pipeline с добавленным в него кастомной трансформации из файла. 
# Затем предскажите значение целевой переменной для наблюдения, которое описывается следующим вектором:

features = np.array([
    [ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]
    ])
# В поле для ответа введите предсказанное значение целевой переменной, округлённое до целого числа.

# Десериализуем pipeline из файла
with open('my_new_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)
    
loaded_pipe.predict(features)

array([173.01985747])

### БИБЛИОТЕКА JOBLIB

In [18]:
import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'regr.joblib')

## ['regr.joblib']

['regr.joblib']

In [19]:
# Загрузим файл заново (загрузка может быть произведена в другом файле с кодом):

# Десериализуем модель из файла
clf_from_jobliv = joblib.load('regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

## True

True

# 3. Практика: pickle

In [20]:
# Ваш коллега Василий обучил модель и теперь просит вас проверить её на ваших данных. 
# Он присылает вам pickle-файл. Загрузите модель, используя модуль pickle.
# 3.1
# При загрузке вывелся секретный код. Введите его в поле ниже.

# Производим десериализацию и извлекаем модель из файла формата pkl
with open('model.pkl', 'rb') as pkl_file:
    model = pickle.load(pkl_file)

model

secret word: skillfactory
how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


LinearRegression(normalize=False, positive=True)

In [23]:
# 3.3
# Теперь необходимо применить модель. Сделайте предсказание для следующего набора фичей: [1, 1, 1, 0.661212487096872]. 
# Введите результат, предварительно округлив его до трёх знаков после точки-разделителя.
f = np.array([[1, 1, 1, 0.661212487096872]])
model.predict(f)

array([0.666])

In [26]:
# 3.4
# У присланной вам модели есть два поля (атрибута) с именами a и b. 
# Создайте из них словарь с такими же именами ключей и значениями, а затем сохраните его в файл с помощью модуля pickle.
my_dict = {'a': model.a, 'b': model.b}

with open('my_dict.pkl', 'wb') as pkl_file:
    pickle.dump(my_dict, pkl_file)

!python hw1_check_ol.py my_dict.pkl

zsh:1: command not found: python


# 4. Сохранение и загрузка моделей: PMML и ONNX-ML

### PREDICTIVE MODEL MARKUP LANGUAGE

In [27]:
!pip3 install nyoka

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.8/303.8 kB 1.1 MB/s eta 0:00:0000:0100:01


In [28]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipeline.pmml")

### OPEN NEURAL NETWORK EXCHANGE

In [ ]:
# 4.6
# В задаче ниже мы обучаем модель sklearn, конвертируем ее в ONNX и делаем инференс через ONNX-runtime.

# Дополните код ниже недостающими элементами:

import onnxruntime as rt 
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from skl2onnx import ___1___
from skl2onnx.common.data_types import ___2___


# загружаем данные
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)

# делаем инференс моделью на тесте
test_pred = model.predict(___4___)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в ONNX-формат
initial_type = [('float_input', ___5___([None, ___6___]))]
model_onnx = ___7___(model, initial_types=initial_type)

# сохраняем модель в файл
with open("model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через ONNX-runtime
sess = rt.___8___("model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx)